In [6]:
import os
from app_decomposer.utils_demo import *
from pprint import pprint

In [23]:
ts = get_job_timeseries_from_file()
#ts = ts.pop("accessPattern")
pprint(ts["volume"])
#ts = ts.pop('processCount')

{'bytesRead': array([       6457,           0,  1090522169,  2055211225,           0,
                 0,           0,           0,           0,           0,
                 0,           0,           0,           0,           0,
       15728643189,           0,           0,           0,           0,
                 0,           0,           0,           0,           0,
                 0,           0,  8965325739, 22491955278,           0,
                 0,           0,           0,           0,           0,
                 0,           0,           0,           0,           0,
                 0,           0,           0,           0,           0,
                 0,           0,           0,           0,         126,
                 0,           0,           0,           0,           0,
                 0,           0,           0,           0,           0,
                 0,           0,          72,           0,           0,
                 0,           0,           0,     

In [ ]:
job1 = {'bytesRead': np.array([       6457,           0,  1090522169,  2055211225]),
        'bytesWritten': np.array([         115404800,  191795200,   0,          0]),
        'timestamp': np.array([1581607510, 1581607515, 1581607520, 1581607525])}

job2 = {'bytesRead': np.array([       2457,           8000,  0,  0]),
        'bytesWritten': np.array([         15404800,  91795200,   0,          0]),
        'timestamp': np.array([1581607520, 1581607525, 1581607530, 1581607535])}

job3 = {'bytesRead': np.array([       6457,           0,  1090522169,  2055211225]),
        'bytesWritten': np.array([         115404800,  191795200,   0,          0]),
        'timestamp': np.array([1581607530, 1581607535, 1581607540, 1581607545])}

In [34]:

jobid=3918
dataset_path = os.path.join(dirname(dirname(os.getcwd())), "dataset_generation", "dataset_generation")
job_files, job_ids, dataset_names = list_jobs(dataset_path)
# create two lists of job_ids where only few elements are differents


print(dataset_path)
print(job_ids)
print(job_files)
csv_file = job_files[job_ids.index("52508")]
df = pd.read_csv(csv_file, index_col=0)  # read the csv file into a dataframe
# print the columns of the dataframe to check if 'operationRead' exists
print(csv_file)
get_job_timeseries_from_file(job_id=7910)


/home_nfs/mimounis/iosea-wp3-recommandation-system/dataset_generation/dataset_generation
['1391', '1335', '1439', '1512', '1476', '1334', '1489', '1336', '1330', '1272', '4949', '4911', '4912', '4951', '4952', '4953', '4954', '4913', '4950', '22557', '6000', '22658', '6970', '6382', '5969', '5932', '22578', '6308', '6376', '22624', '6264', '507740', '507738', '465283', '542439', '465155', '480598', '486693', '507742', '457344', '488287', '480600', '469883', '7584', '7910', '11638', '3564', '4281', '7518', '7256', '2537', '2301', '4821', '7851', '6168', '107815', '107939', '105836', '109567', '107187', '107416', '108010', '101329', '107998', '107251', 'dataset', '5189', '5296', '5171', '5179', '5170', '5173', '5288', '5186', '5185', '5183', '5180', '5286', '5176', '5285', '5295', '5293', '5190', '5193', '5294', '5181', '5175', '5168', '5184', '5188', '5282', '5291', '5292', '5178', '5191', '5289', '5182', '5287', '5281', '5283', '5169', '5192', '5284', '5172', '5187', '5194', '5290', '5

KeyError: "None of [Index(['operationRead'], dtype='object')] are in the [columns]"